In [2]:
#import needed modules
import pandas as pd
import numpy as np
import geopandas
import geopy
import io
import folium
import folium.plugins as plugins
import nominatim

In [10]:
#uploaded = ('/Users/mark/Geo/Address Centroid/TestAddress.xlsx')
df = pd.read_excel('/Users/mark/Geo/Addresses Centroid/TestAddress.xlsx')

#Preview df
df

,Address,Coords
0,20652 Cutwater Place,NaN
1,20622 Muddy Harbour Square,NaN
2,47621 Sandbank Square,NaN
3,46390 Hampshire Station Dr,NaN


In [21]:
#Calculate coords from address
locator = geopy.Nominatim(user_agent="myGeocoder")
#location = locator.geocode("Champ de Mars, Paris, France") #Works for Eifle Tower
location = locator.geocode("Chantilly, Virginia, United States of America") #Works

GeoCoord = []
for address in df['Address']:
    GeoCoord = locator.geocode(address)

    #Translate address to geocoords
Addy = ("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 38.8788355, Longitude = -77.40221718011088


In [27]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df.dropna(how='all')
del df['Coords']
df

,Address,location,point,latitude,longitude,altitude
0,20652 Cutwater Place,"(20652, Cutwater Place, Cascades, Loudoun Coun...","(39.0444875, -77.39898296254646, 0.0)",39.044488,-77.398983,0.0
1,20622 Muddy Harbour Square,None,None,NaN,NaN,NaN
2,47621 Sandbank Square,"(47621, Sandbank Square, Lowes Island, Loudoun...","(39.04046057979135, -77.35091644583981, 0.0)",39.040461,-77.350916,0.0
3,46390 Hampshire Station Dr,"(46390, Hampshire Station Drive, Cascades, Lou...","(39.048063049999996, -77.3899942077798, 0.0)",39.048063,-77.389994,0.0


In [28]:
pts = []
lat = []
long = []
alt = []

for point in df['point']:
    if point is not None:
        pts.append(point)
'''
for latitude in df['latitude']:
  if latitude is not None:
    lat.append(latitude)
for longitude in df['longitude']:
  if longitude is not None:
    long.append(longitude)
for altitude in df['altitude']:
  if altitude is not None:
    alt.append(altitude)

'''
df3 = pd.DataFrame()
#df3['latitude'] = lat
df3['points'] = pts

#df3['long'] = long
#df3['alt'] = alt
df3

,points
0,"(39.0444875, -77.39898296254646, 0.0)"
1,"(39.04046057979135, -77.35091644583981, 0.0)"
2,"(39.048063049999996, -77.3899942077798, 0.0)"


In [29]:
folium_map = folium.Map(width=1500,height=1000, location=[39.040471311943094, -77.35664071247609],
                        zoom_start=12,
                        tiles='OpenStreetMap')


plugins.FastMarkerCluster(data=list(df3['points'])).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [30]:
#Map Plot 1

map1 = folium.Map(
    location=[38.8788355,-77.40221718011088],
    tiles='cartodbpositron',
    zoom_start=12,
)
#map1 = df.replace('',np.nan)
#map1 = df.dropna(axis="rows", how="any")
df3.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)


map1

KeyError: 'latitude'

In [12]:
#coords to list
codes = []
for i in df['Coords']:
    codes.append(i)
codes

[nan, nan, nan, nan]

In [18]:
#Show map and set focal starting coord
reston = folium.Map(location=[39.04245261101, -77.35889503805], zoom_start=13)
reston